In [2]:
import pickle
import os
import sys
import datetime
import pandas as pd
import numpy as np
import nltk.data
from gensim.models import Word2Vec
import multiprocessing
from copy import deepcopy
from sklearn.metrics import roc_auc_score    
from sklearn.cross_validation import StratifiedKFold

import random
random.seed(1)

In [3]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Make sure that df was built the same way as for other datasets.

In [4]:
df = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/pickled_objects/dfJournals_trans_categories.p', 'rb'))

In [5]:
print df.shape
print df.columns.values
df.columns = ['label', 'title', 'abstract', 'qualifier_terms', 'descriptor_terms']

(97048, 5)
['sentiment' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']


In [6]:
print df.shape
print df.columns.values
print df['label'].unique()

(97048, 5)
['label' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms']
['T0' 'T1/T2' 'T3/T4']


In [7]:
df[0:10]

,label,title,abstract,qualifier_terms,descriptor_terms
0,T0,Quantification of the relative contributions o...,Amyotrophic lateral sclerosis (ALS) and fronto...,,
1,T0,Ubiquitin Specific Protease 36 (USP36) control...,Ubiquitination of the TrkA neurotrophin recept...,,
2,T0,Functional Diversity of Cytotoxic tRNase/immun...,Contact-dependent growth inhibition (CDI) is a...,,
3,T0,Identification of a distinct substrate binding...,The type III secretion system (T3SS) effector ...,,
4,T0,Role of chondroitin sulfate (CS) modification ...,Protein tyrosine phosphatase receptor type Z (...,,
5,T0,Chemically programmed bispecific antibodies in...,Chemically programmed bispecific antibodies (b...,,
6,T0,Non-mutagenic Suppression of Enterocyte Ferrop...,Iron transfer across the basolateral membrane ...,,
7,T0,Structural plasticity of cholesteryl ester tra...,Cholesteryl ester transfer protein (CETP) medi...,,
8,T0,Identification of a Membrane-Bound Prepore Spe...,Pore-forming toxins (PFT) are cytolytic protei...,,
9,T0,EspR-dependent ESAT-6 secretion of Mycobacteri...,Attenuation of M. bovis BCG strain is related ...,,


## Apply word2vec inversion

In [8]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

## Combined method (only has to be run once instead of 3 times)

In [9]:
df_val = pickle.load(open('/mnt/mypartition/pubmed_nlp_research/C_article_replication_and_translational_classification/df.p', 'rb'))

In [10]:
def buildModels(train1, test1, df_val, docs_val, docs, basemodel, label):
    
    #add boolean labels
    train1['boolean_label'] = 1
    for row in train1.iterrows():
        if row[1]['label'] != label:
            train1.set_value(row[0], 'boolean_label', 0)
            
    test1['boolean_label'] = 1
    for row in test1.iterrows():
        if row[1]['label'] != label:
            test1.set_value(row[0], 'boolean_label', 0)
    
    # ****** Split the training set into clean sentences
    #
    sentences_pos = []  # Initialize an empty list of sentences
    sentences_neg = []  # Initialize an empty list of sentences

    #here change to include all journal name labels of positive and negative
    inxs_pos = np.where(train1['boolean_label'] == 1)[0].tolist()
    inxs_neg = np.where(train1['boolean_label'] == 0)[0].tolist()

    for inx in inxs_pos:
        review = train1["abstract"].iloc[inx]
        sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    for inx in inxs_neg:
        review = train1["abstract"].iloc[inx]
        sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
         
    #train models
    models = [deepcopy(basemodel) for y in range(2)]
    models[0].train(sentences_neg, total_examples=len(sentences_neg) )
    models[1].train(sentences_pos, total_examples=len(sentences_pos) )
    
    probs = docprob(docs,models).astype(object)
    
    predictions = np.ones((probs.shape[0]))

    predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

    score = roc_auc_score(test1["boolean_label"], predictions)
    
    #------------------------------------------------------------------------------------------------------------    
    print "Testing on external validation set..."

    df_val['boolean_label'] = 1
    for row in df_val.iterrows():
        if row[1]['label'] != label:
            df_val.set_value(row[0], 'boolean_label', 0)

    print "Making sentiment label predictions..."
    probs_val = docprob(docs_val,models).astype(object)

    predictions_val = np.ones((probs_val.shape[0]))

    predictions_val[np.where(probs_val.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

    score_val = roc_auc_score(df_val["boolean_label"], predictions_val)

    return score, models, score_val

In [13]:
a = datetime.datetime.now().replace(microsecond=0)

y = df['label'].values
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=10)

i = 1
avgAll = []
avgT0 = []
avgT1T2 = []
avgT3T4 = []

avgAll_val = []
avgT0_val = []
avgT1T2_val = []
avgT3T4_val = []

for train_index, test_index in skf:

    print("ROUND", i)
    i = i + 1
    #use the indexes to subset the df pandas dataframe
    train1, test1 = df.iloc[train_index], df.iloc[test_index]

    print "Parsing sentences from training set"

    # ****** Split the labeled and unlabeled training sets into clean sentences
    #
    sentences = []  # Initialize an empty list of sentences

    for j in range(len(train1)):
        review = train1["abstract"].iloc[j]
        sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

    print "Building and training w2v models"
    ## create a w2v learner 
    basemodel = Word2Vec(
        workers=multiprocessing.cpu_count(), # use your cores
        iter=3, # iter = sweeps of SGD through the data; more is better
        hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
        )
    basemodel.build_vocab(sentences) 

    print "Parsing test sentences"
    # read in the test set as a list of a list of words

    docs = []
    for k in range(len(test1)):
        review = test1["abstract"].iloc[k]
        sents = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        docs.append(sents)
        
    print "Parsing validation sentences"
    docs_val = []
    for k in range(len(df_val)):
        review = df_val["abstract"].iloc[k]
        sents = KaggleWord2VecUtility.review_to_sentences(review, tokenizer)
        docs_val.append(sents)

    print "Scoring test set"
    tupT0 = buildModels(train1, test1, df_val, docs_val, docs, basemodel, 'T0')
    scoreT0 = tupT0[0]
    print scoreT0
    avgT0.append(scoreT0)
    avgAll.append(scoreT0)
    modelsT0 = tupT0[1]
    score_val = tupT0[2]
    avgT0_val.append(score_val)
    avgAll_val.append(score_val)

    tupT1T2 = buildModels(train1, test1, df_val, docs_val, docs, basemodel, 'T1/T2')
    scoreT1T2 = tupT1T2[0]
    print scoreT1T2
    avgT1T2.append(scoreT1T2)
    avgAll.append(scoreT1T2)
    modelsT1T2 = tupT1T2[1]
    score_val = tupT1T2[2]
    avgT1T2_val.append(score_val)
    avgAll_val.append(score_val)

    tupT3T4 = buildModels(train1, test1, df_val, docs_val, docs, basemodel, 'T3/T4')
    scoreT3T4 = tupT3T4[0]
    print scoreT3T4
    avgT3T4.append(scoreT3T4)
    avgAll.append(scoreT3T4)
    modelsT3T4 = tupT3T4[1]
    score_val = tupT3T4[2]
    avgT3T4_val.append(score_val)
    avgAll_val.append(score_val)

print ""
print "---------------------------------------------------------------------------------"
print "TEST SET"
print "Average for T0 model:"
avgT0 = sum(avgT0) / float(len(avgT0))
print avgT0
print "Average for T1/T2 model:"
avgT1T2 = sum(avgT1T2) / float(len(avgT1T2))
print avgT1T2
print "Average for T3/T4 model:"
avgT3T4 = sum(avgT3T4) / float(len(avgT3T4))
print avgT3T4
print ""
print "Average across all models:"
print sum(avgAll) / float(len(avgAll))

print ""
print "---------------------------------------------------------------------------------"
print "VALIDATION SET"
print "Average for T0 model:"
avgT0_val = sum(avgT0_val) / float(len(avgT0_val))
print avgT0_val
print "Average for T1/T2 model:"
avgT1T2_val = sum(avgT1T2_val) / float(len(avgT1T2_val))
print avgT1T2_val
print "Average for T3/T4 model:"
avgT3T4_val = sum(avgT3T4_val) / float(len(avgT3T4_val))
print avgT3T4_val
print ""
print "Average across all models:"
print sum(avgAll_val) / float(len(avgAll_val))
    
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

('ROUND', 1)
Parsing sentences from training set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "https://clinicaltrials.gov/ct2/show/NCT02351037." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' 

Building and training w2v models
Parsing test sentences


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.fp7-improved.eu/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://clinicaltrials.gov/ct2/show/NCT00679367." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/nlp-vm/anacond

Parsing validation sentences
Scoring test set


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Testing on external validation set...
Making sentiment label predictions...
0.93941987441
Testing on external validation set...
Making sentiment label predictions...
0.810679837271
Testing on external validation set...
Making sentiment label predictions...
0.875031668564
('ROUND', 2)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
Parsing validation sentences
Scoring test set
Testing on external validation set...
Making sentiment label predictions...
0.941171686948
Testing on external validation set...
Making sentiment label predictions...
0.813878305311
Testing on external validation set...
Making sentiment label predictions...
0.871518711923
('ROUND', 3)
Parsing sentences from training set
Building and training w2v models
Parsing test sentences
Parsing validation sentences
Scoring test set
Testing on external validation set...
Making sentiment label predictions...
0.941924860634
Testing on external validation set...
Making sentiment label p

## Performance:
Overall Average- 0.8751

T0 class- 0.9411 (0.94, 0.94)

T1/T2 class- 0.8122 (0.81, 0.81)

T3/T4 class- 0.8719 (0.87, 0.87)

*much consistency across folds

In [14]:
pickle.dump(modelsT0, open('/mnt/mypartition/reproducing_methods/w2vmodelsJournalsT0.p', 'wb'))
pickle.dump(modelsT1T2, open('/mnt/mypartition/reproducing_methods/w2vmodelsJournalsT1T2.p', 'wb'))
pickle.dump(modelsT3T4, open('/mnt/mypartition/reproducing_methods/w2vmodelsJournalsT3T4.p', 'wb'))

NameError: name 'models' is not defined

### External validation on paper's dataset

In [16]:
#check that column names are the same
print df_val.shape
print df_val.columns.values
print df_val['label'].unique()
df_val[0:10]

(354, 6)
['pmid' 'title' 'abstract' 'qualifier_terms' 'descriptor_terms' 'label']
['T0' 'T1/T2' 'T3/T4' 'TX']


,pmid,title,abstract,qualifier_terms,descriptor_terms,label
0,23195993,Gene expression profiles in peripheral blood m...,Occupational exposure to nickel (Ni) is associ...,genetics metabolism epidemiology drug effects ...,Adult Asian Continental Ancestry Group Biomark...,T0
1,25077433,SOX2 is a cancer-specific regulator of tumour ...,Although the principles that balance stem cell...,genetics metabolism pathology metabolism patho...,"Animals Carcinoma, Squamous Cell Cell Line, Tu...",T0
2,24107601,Imaging and cerebrospinal fluid biomarkers in ...,The pathophysiological process of Alzheimer's ...,cerebrospinal fluid genetics radionuclide imag...,"Aged Aged, 80 and over Alzheimer Disease Amylo...",T0
3,24891029,Preliminary evidence of cognitive and brain ab...,To ascertain whether pediatric obesity without...,pathology physiopathology physiopathology,Adolescent Attention Brain Cognition Executive...,T0
4,21691448,Obese Adolescents with Type 2 Diabetes Mellitu...,The rates of type 2 diabetes (T2DM) continue t...,,,T0
5,22765222,Diversity of 5S rRNA genes within individual p...,We examined intragenomic variation of paralogo...,chemistry classification genetics chemistry ge...,"Bacteria DNA, Ribosomal Databases, Nucleic Aci...",T0
6,24401686,Myoinositol and glutamate complex neurometabol...,To obtain quantitative neurometabolite measure...,analogs & derivatives metabolism metabolism me...,Adult Aspartic Acid Brain Injuries Case-Contro...,T0
7,22914093,Antibiotics in early life alter the murine col...,Antibiotics administered in low doses have bee...,drug effects physiology administration & dosag...,Adiposity Age Factors Animals Anti-Bacterial A...,T0
8,23426830,Elevated serum anti-Müllerian hormone in adole...,Serum anti-Müllerian hormone (AMH) is linked t...,blood pathology ultrasonography blood ultrason...,Adolescent Anti-Mullerian Hormone Child Female...,T0
9,24344399,Association of obesity-mediated insulin resist...,The hypothalamus is important in hunger and me...,blood analysis anatomy & histology blood blood...,Adolescent Adult Brain-Derived Neurotrophic Fa...,T0


In [ ]:
#Optional: Remove 'TX' scores to test performance
df_val = df_val[df_val['label'] != 'TX']
print len(df_val)

### Import KaggleWord2VecUtility since didn't work from file

In [12]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords


class KaggleWord2VecUtility(object):
    """KaggleWord2VecUtility is a utility class for processing raw HTML text into segments for further learning"""

    @staticmethod
    def review_to_wordlist( review, remove_stopwords=False ):
        # Function to convert a document to a sequence of words,
        # optionally removing stop words.  Returns a list of words.
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text()
        #
        # 2. Remove non-letters
        review_text = re.sub("[^a-zA-Z]"," ", review_text)
        #
        # 3. Convert words to lower case and split them
        words = review_text.lower().split()
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]
        #
        # 5. Return a list of words
        return(words)
    
    @staticmethod
    def review_to_words( review, remove_stopwords=False ):
        # Function to convert a raw review to a string of words
        # The input is a single string (a raw movie review), and 
        # the output is a single string (a preprocessed movie review)
        #
        # 1. Remove HTML
        review_text = BeautifulSoup(review).get_text() 
        #
        # 2. Remove non-letters        
        review_text = re.sub("[^a-zA-Z]", " ", review_text) 
        #
        # 3. Convert to lower case, split into individual words
        words = review_text.lower().split()                             
        #
        # 4. Optionally remove stop words (false by default)
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            words = [w for w in words if not w in stops]   
        #
        # 6. Join the words back into one string separated by space, 
        # and return the result.
        return( " ".join( words ))   

    # Define a function to split a review into parsed sentences
    @staticmethod
    def review_to_sentences( review, tokenizer, remove_stopwords=False ):
        # Function to split a review into parsed sentences. Returns a
        # list of sentences, where each sentence is a list of words
        #
        # 1. Use the NLTK tokenizer to split the paragraph into sentences
        raw_sentences = tokenizer.tokenize(review.decode('utf8').strip())
        #
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call review_to_wordlist to get a list of words
                sentences.append( KaggleWord2VecUtility.review_to_wordlist( raw_sentence, \
                  remove_stopwords ))
        #
        # Return the list of sentences (each sentence is a list of words,
        # so this returns a list of lists
        return sentences